In [1]:
import pandas as pd
import numpy as np
import os, sys, re
from StringIO import StringIO
import itertools

In [2]:
sys.path.append("./static/python/")
sys.path.append("./../metaprot/sql/")

In [72]:
connection = db_config.Connect(echo=True, testing=False, do_logging=False)
protein_ans_list = ["P35997", "P38711", "P35997", "P38711", "B2RID1", "belk_c_455_5138"]
# protein_ans_list = ["B2RID1"]

In [73]:
stmt = """SELECT protein_2_function.an, protein_2_function.function
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
;""" #AND ontologies.parent='GO:0003674'
session = connection.get_session()
result = session.execute(stmt).fetchall()
session.close()

2016-09-09 15:24:36,697 INFO sqlalchemy.engine.base.Engine select version()
2016-09-09 15:24:36,698 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 15:24:36,699 INFO sqlalchemy.engine.base.Engine select current_schema()
2016-09-09 15:24:36,700 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 15:24:36,701 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-09-09 15:24:36,701 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 15:24:36,702 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-09-09 15:24:36,703 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 15:24:36,704 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2016-09-09 15:24:36,705 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 15:24:36,706 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-09 15:24:36,707 INFO sqlalchemy.engine.base.Engine SELECT protein_2_function.an, protein_2_function.function
FROM pro

In [47]:
# result

In [95]:
import userinput
reload(userinput)
import query, db_config
reload(query)
reload(db_config)

<module 'db_config' from './../metaprot/sql/db_config.pyc'>

In [108]:
an = "P35997"
d = query.get_association_dict(connection, protein_ans_list=[an], 
                           function_type="GO", limit_2_parent="BP", 
                           basic_or_slim="basic", backtracking=True)
print len(d[an])
x = d[an]

2016-09-09 15:43:22,834 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-09 15:43:22,835 INFO sqlalchemy.engine.base.Engine SELECT protein_2_function.an, ontologies.child, ontologies.parent
FROM protein_2_function
INNER JOIN functions ON protein_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
WHERE protein_2_function.an IN('P35997')
AND functions.type='GO'
AND ontologies.parent='GO:0008150'
;
2016-09-09 15:43:22,836 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 15:43:23,252 INFO sqlalchemy.engine.base.Engine ROLLBACK
2016-09-09 15:43:23,253 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-09 15:43:23,254 INFO sqlalchemy.engine.base.Engine SELECT protein_2_og.an, og_2_function.function
FROM protein_2_og
INNER JOIN og_2_function ON protein_2_og.og=og_2_function.og
INNER JOIN functions ON og_2_function.function=functions.an
INNER JOIN ontologies ON ontologies.child=functions.an
WHERE protein_2_og.an IN('P35997')
AND function

In [129]:
# all directly annotated GO terms for "P35997"
x = """3782218	P35997	GO:0000028
3782219	P35997	GO:0000462
3782220	P35997	GO:0002181
3782221	P35997	GO:0003735
3782222	P35997	GO:0006412
3782223	P35997	GO:0022627
3782224	P35997	GO:0046872"""
go_list = []
for ele in x.split("\n"):
    go_list.append(ele.split("\t")[-1])
print len(go_list)
go_list    

7


['GO:0000028',
 'GO:0000462',
 'GO:0002181',
 'GO:0003735',
 'GO:0006412',
 'GO:0022627',
 'GO:0046872']

In [130]:
x = sorted(x)
x = sorted([str(ele) for ele in x])
x == go_all

False

In [131]:
stmt = """SELECT ontologies.child FROM ontologies
WHERE ontologies.child IN({})
AND ontologies.parent = 'GO:0008150';""".format(str(go_list)[1:-1])
session = connection.get_session()
result = session.execute(stmt).fetchall()
session.close()
result

2016-09-09 16:28:56,201 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-09 16:28:56,202 INFO sqlalchemy.engine.base.Engine SELECT ontologies.child FROM ontologies
WHERE ontologies.child IN('GO:0000028', 'GO:0000462', 'GO:0002181', 'GO:0003735', 'GO:0006412', 'GO:0022627', 'GO:0046872')
AND ontologies.parent = 'GO:0008150';
2016-09-09 16:28:56,202 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 16:28:56,208 INFO sqlalchemy.engine.base.Engine ROLLBACK


[(u'GO:0000462',), (u'GO:0002181',), (u'GO:0000028',), (u'GO:0006412',)]

In [132]:
# all parents of go_list
# go_list = ['GO:0000028', 'GO:0000462', 'GO:0002181', 'GO:0006412', ]
stmt = """SELECT ontologies.child, ontologies.parent FROM ontologies AS ont_1
INNER JOIN ontologies ON 
WHERE ontologies.child IN({});""".format(str(go_list)[1:-1])
#AND ontologies.parent IN('GO:0005575', 'GO:0008150', 'GO:0003674')
# ontologies.child, ontologies.parent FROM ontologies
# AND ontologies.parent = 'GO:0008150'
# GO:0005575
# GO:0008150
# GO:0003674
'GO:0005575', 'GO:0008150', 'GO:0003674'
session = connection.get_session()
result = session.execute(stmt).fetchall()
session.close()
# go_all = []
# for res in result:
#     go_all += list(res)
# go_all = sorted(set(go_all))
# print len(go_all)
# go_all
result

2016-09-09 16:28:57,300 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-09 16:28:57,301 INFO sqlalchemy.engine.base.Engine SELECT ontologies.child, ontologies.parent FROM ontologies
WHERE ontologies.child IN('GO:0000028', 'GO:0000462', 'GO:0002181', 'GO:0003735', 'GO:0006412', 'GO:0022627', 'GO:0046872');
2016-09-09 16:28:57,302 INFO sqlalchemy.engine.base.Engine {}
2016-09-09 16:28:57,303 INFO sqlalchemy.engine.base.Engine ROLLBACK


[(u'GO:0022627', u'GO:0005575'),
 (u'GO:0022627', u'GO:0032991'),
 (u'GO:0022627', u'GO:0015935'),
 (u'GO:0022627', u'GO:0044464'),
 (u'GO:0022627', u'GO:1990904'),
 (u'GO:0022627', u'GO:0030529'),
 (u'GO:0022627', u'GO:0044446'),
 (u'GO:0022627', u'GO:0044444'),
 (u'GO:0022627', u'GO:0044445'),
 (u'GO:0022627', u'GO:0044424'),
 (u'GO:0022627', u'GO:0044391'),
 (u'GO:0022627', u'GO:0044422'),
 (u'GO:0000462', u'GO:0034470'),
 (u'GO:0000462', u'GO:0044238'),
 (u'GO:0000462', u'GO:0009987'),
 (u'GO:0000462', u'GO:0006725'),
 (u'GO:0000462', u'GO:0090304'),
 (u'GO:0000462', u'GO:0034641'),
 (u'GO:0000462', u'GO:0006807'),
 (u'GO:0000462', u'GO:0008150'),
 (u'GO:0000462', u'GO:0008152'),
 (u'GO:0000462', u'GO:0034660'),
 (u'GO:0000462', u'GO:1901360'),
 (u'GO:0000462', u'GO:0046483'),
 (u'GO:0000462', u'GO:0016070'),
 (u'GO:0000462', u'GO:0006139'),
 (u'GO:0000462', u'GO:0016072'),
 (u'GO:0000462', u'GO:0044260'),
 (u'GO:0000462', u'GO:0030490'),
 (u'GO:0000462', u'GO:0044237'),
 (u'GO:000

In [ ]:
s = StringIO()

In [ ]:
f = u'BUBU;BUBU2;BUBU3\r\nAUBU;AUBU2;AUBU3\r\nP40545\r\nP09201\r\nP14066\r\nP25040\r\nP20050\r\nP40327\r\nQ01939\r\nQ01939'
b = u'BUBU\t\r\nBUBU\t\r\nP00359\t11,07751325\r\nP00925\t10,96978388\r\nP02994\t10,76849751\r\nP06169\t10,74716312\r\nP00560\t10,74614359\r\nP00950\t10,74149039\r\nP00549\t10,70474233\r\nP00330\t10,6690378\r\nAIAIAI\r\nUUUU;IIII'

In [ ]:
# zip(f.split("\r\n"), b.split("\r\n"))

In [ ]:
s = StringIO()
s.write('foreground\tbackground\tintensity\r')
for a, b in itertools.izip_longest(f.split("\r\n"), b.split("\r\n"), fillvalue="\t"):
    s.write(a + "\t" + b.strip() + "\r")

In [ ]:
s.tell()

In [ ]:
s.seek(0)

In [ ]:
s.tell()

In [ ]:
set(["all_GO", "BP", "CP", "MF"])

In [ ]:
{"BP", "CP", "MF"}

In [ ]:
s.read()

In [ ]:
s.split("\r")

In [ ]:
x = 'intensity\tbackground\tforeground\r\tBUBU\t"BUBU;BUBU2;BUBU3"\r\tBUBU\tAUBU;AUBU2;AUBU3\r11,07751325\tP00359\tP40545\r10,96978388\tP00925\tP09201\r10,76849751\tP02994\tP14066\r10,74716312\tP06169\tP25040\r10,74614359\tP00560\tP20050\r10,74149039\tP00950\tP40327\r10,70474233\tP00549\tQ01939\r10,6690378\tP00330\tQ01939\r10,64206863\tP05743\tP38142\r10,64061061\t"P02407;P14127"\tQ01939\r10,63619693\t"P0C2H6;P0C2H7"\tP53549\r10,6231769\t"P35997;P38711"\tP53549\r10,61589747\tP29453\tP39959\r10,59787062\tO14455\tP38787\r10,59595893\tP04456\tP38787\r10,5819724\tP14540\tP12684\r10,58108453\t"P0CX84;P0CX85"\tP12684\r10,58103895\tP05755\tP38122\r10,56810781\t"P0CX31;P0CX32"\tP38122\r10,54364595\t"P0CX55;P0CX56"\tP0CX47\r10,5379072\tP38061'

In [ ]:
x.split("\r")

In [ ]:
s.write("bubu")

In [ ]:
s.tell()

In [ ]:
fn = r"/Users/dblyon/CloudStation/CPR/BTW_GO/UserInput.txt"
fn = r"/Users/dbl/modules/cpr/agotool/static/data/exampledata/exampledata_yeast.txt"
# fn = r"/Users/dblyon/modules/cpr/agotool/static/data/exampledata/refactorUserinput.txt"
df = pd.read_csv(fn, sep='\t', decimal=",")#, converters={"background_int": float, "background_an": str, "foreground_an": str})

In [ ]:
# fh = open(fn, "r").read()
ui = userinput.Userinput(fn=fn, foreground_string=None, background_string=None,
col_foreground='foreground', col_background='background', col_intensity='intensity',
num_bins=100, decimal='.', method="abundance_correction")

In [ ]:
ui.background

In [ ]:
cond = ui.foreground["intensity"] > -1
bins = pd.cut(ui.foreground.loc[cond, "intensity"], bins=100, retbins=True)[1]
bins = [-1] + list(bins)

In [ ]:
ui.foreground

In [ ]:
for x in ui.iter_bins():
#     print wf, bins
#     print bg, fg
#     print ans, cf
    print x
    print

In [ ]:
mbps = 30.0
gb = mbps / 1000.0
min_ = gb * 60
hour = min_ * 60
print min_
print hour
tb = 2.0
(tb * 1000) / hour

In [ ]:
bins = pd.cut(ui.foreground["intensity"], bins=100, retbins=True)[1]
groups_fg = ui.foreground.groupby(pd.cut(ui.foreground["intensity"], bins=bins))
groups_bg = ui.background.groupby(pd.cut(ui.background["intensity"], bins=bins))

In [ ]:
# ui.background[ui.background["intensity"] <= 0]

In [ ]:
for group_fg, group_bg in zip(groups_fg, groups_bg):
    bins_in_group = group[0]
    pg_fg = group_fg[1]["foreground"]
    pg_bg = group_bg[1]["background"]
    print pg_fg, len(pg_fg)
    print pg_bg, len(pg_bg)
    print 
    break

In [ ]:
# for group in groups:
# #     bins_in_group = group[0]
# #     pg_in_group = group[1]["foreground"]
# #     print bins_in_group
# #     print pg_in_group
#     print group
#     print 

In [ ]:
hist, bins = np.histogram(ui.foreground["intensity"], bins=100)

In [ ]:
bins # (-1.0121, -0.879]

In [ ]:
hist

In [ ]:
10.71518785 -  10.83596299

In [ ]:
9.62821166 -   9.74898679

In [ ]:
df.loc[0, "background_int"]

In [ ]:
# np.histogram(df["background_int"], bins=10)
np.histogram(df.loc[pd.notnull(df["background_int"]), "background_int"], bins=10)

In [ ]:
ui.background_df

In [ ]:
# import math

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
# arr = list(np.arange(10))
# arr += [2.3]*8
# hist, bin_edges = np.histogram(arr, bins=10, density=False)
# plt.hist(arr, bins=bin_edges)  # plt.hist passes it's arguments to np.histogram
# plt.show()

In [ ]:
# good stuff

In [ ]:
# foreground proteinGroups with intensities
intensity_foreground = []
for proteinGroup in ui.foreground_ser:
    an_first_in_proteinGroup = proteinGroup.split(";")[0]
    intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     try:      
#         intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     except KeyError:
#         # can't occur, since defaultdict
#         raise StopIteration

In [ ]:
ui.foreground_df = pd.DataFrame(data=intensity_foreground, columns=["Int"])
ui.foreground_df["AN"] = ui.foreground_ser

In [ ]:
df = ui.foreground_df
df

In [ ]:
1.5*30

In [ ]:
# create bins
# which proteinGroups from background are within bin

In [ ]:
num_bins = 100
bins = np.linspace(df["Int"].min(), df["Int"].max(), num_bins)
groups = df.groupby(pd.cut(df["Int"], bins))

In [ ]:
# for group in groups:
#     print group
#     print